# Assignment 8: Neural Networks

Only use the already imported library `numpy` and the Python standard library. For the evaluation you may also use scikit-learn (`sklearn`) and `matplotlib`. Make sure that the dataset `airfoil_self_noise.csv` is in the same directory as the notebook.

List your team members (name and immatriculation number) and indicate whether you are a B.Sc. Data Science or other group in the following cell:

- Kuang-Yu Li, st169971@stud.uni-stuttgart.de, 3440829 
- Ya-Jen Hsu, st169013@stud.uni-stuttgart.de, 3449448 
- Gabriella Ilena, st169935@stud.uni-stuttgart.de, 3440942

In [32]:
import numpy as np

def load_dataset(path):
    from sklearn.model_selection import train_test_split
    
    data = np.genfromtxt(path)
    X, y = data[:, :5], data[:, 5]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)

    return X_train, X_test, y_train, y_test
    

X_train, X_test, y_train, y_test = load_dataset('airfoil_self_noise.csv')

## Task 3: Feedforward Neural Network: Programming

In this task, you will implement a feedforward neural network for regression. The hyperparameters of the model are:
- `input_dim`: The dimension of the input vector.
- `output_dim`: The dimension of the output vector.
- `width`: The dimension of each hidden layer.
- `depth`: The number of hidden layers. For B.Sc. Data Science students, this parameter is constant with a value of 1.
- `learning_rate`: The learning rate for gradient descent.
- `epochs`: The number of epochs/iterations performed during training.

B.Sc. Data Science only have to implement for a single hidden layer, i.e. `depth = 1`. All other students have to implement the network for any `depth >= 1`.

The activation function for each hidden layer is ReLU (g(x) = max(0, x)). The output layer uses the identity as activation, since our objective is regression.

You have to implement the `FeedforwardNeuralNetworkRegressor`.

The `__init__` method initializes the network.
Initialize each weight and bias randomly with a standard Gaussian distribution using the numpy function `numpy.random.normal` with default parameters.

The `fit` method trains the network.
Use backpropagation with gradient descent similar to Task 2.
Use the whole training data set for each training epoch.
Use the mean squared error as loss function.

The `predict` method computes the forward-pass of the network.

Evaluate your classifier on the test data with the mean squared error and compare your results to your linear regression model from assignment 3. Try out different hyper-parameters and compare the results. You may want to normalize your input and output data for better performance.

In [74]:
class FeedforwardNeuralNetworkClassifier(object):
    def __init__(self, input_dim, output_dim, width, depth, learning_rate, epochs):
        # Add your code, such as initialization of weights here.
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.width = width
        self.depth = depth
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        # Initialize the weights and biases for each layer randomly based on the normal distribution
        # General rule for the dimensions:
        # Weights have shape (n, m), where n is the number of output neurons and m is the number of input neurons
        # Biases have shape (n, 1)
        np.random.seed(5) # Seed to get reproducable results
        self.w = list()
        self.b = list()

        self.w.append(np.random.normal(size=(width, input_dim))) # Input layer
        self.b.append(np.random.normal(size=(width,1)))
        
        for i in range(depth): # Hidden layers
          self.w.append(np.random.normal(size=(width, width)))
          self.b.append(np.random.normal(size=(width, 1)))
        
        self.w.append(np.random.normal(size=(output_dim, width))) # Output layer
        self.b.append(np.random.normal(size=(output_dim, 1)))
        
        self.params = {'w': self.w, 'b': self.b}

    def relu(self, X):
        return np.maximum(0, X)

    def relu_derivative(self, X):
        X[X<0] = 0.
        X[X>0] = 1.
        return X

    def fit(self, X, y):
        # Implement your training here

        for i in range(self.epochs):
          # Forward propagation
          # Output = activation_function(W.X + b)
          outputs = dict()
          outputs[0] = self.relu(np.dot(self.params['w'][0], X.T) + self.params['b'][0])  # Output of 1st hidden layer. Transpose X to get column vector representation
            
          for j in range(self.depth):
            outputs[j+1] = self.relu(np.dot(self.params['w'][j+1], outputs[j]) + self.params['b'][j+1])  # Output of 2nd until the last hidden layer
            
          outputs[self.depth+1] = np.dot(self.params['w'][self.depth+1], outputs[self.depth]) + self.params['b'][self.depth+1]  # Output of the last layer

          # Calculate loss
          y.shape = outputs[self.depth+1].shape # Since the output is given as a row vector, need to ensure it has the same shape as the label vector to prevent wrong broadcasting
          loss = np.square(outputs[self.depth+1] - y).mean()
          assert outputs[self.depth+1].shape == y.shape, "Vectors need to have the same shape"

          # Loss gradient
          dloss = outputs[self.depth+1] - y
    
          # Back-propagation
          grads = dict()

          # From the output layer
          grads[self.depth+1] = np.dot(dloss, self.relu_derivative(outputs[self.depth]).T)
          self.params['w'][self.depth] = self.params['w'][self.depth] - self.learning_rate*(grads[self.depth+1]) # Update output weights
          self.params['b'][self.depth] = self.params['b'][self.depth] - self.learning_rate*(dloss)    

    def predict(self, X):
        # Implement your prediction here.
        return None

# Small testing
x = np.array([[1., 2., 3.],
              [4., 5., 6 ]])
y = np.array([0, 1])
test = FeedforwardNeuralNetworkClassifier(x.shape[1], 1, 3, 2, 0.1, 2) # Output dimension is always 1 for regression?
test.fit(x, y)


In [ ]:
# Implement your training and evaluation here.